In [ ]:
# Célula 1: Imports e Configuração do Caminho (Path)

import pandas as pd
import sys
import os

# --- Adiciona a pasta 'src' ao path do Python ---
project_root = os.path.abspath(os.path.join('..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
# -----------------------------------------------

# Importa as funções de dados e a NOVA função de churn
from data.make_dataset import get_historical_data
from features.build_churn_labels import create_churn_labels

print("Módulos importados com sucesso.")

In [ ]:
# Célula 2: Carregar Features (FASE 2) e Labels (FASE 5)

print("--- Carregando dados... ---")

# 1. Carregar 'features_clientes.csv' (Resultado da FASE 2)
features_path = os.path.join(project_root, 'dados', 'processed', 'features_clientes.csv')
try:
    df_features = pd.read_csv(features_path, index_col='id_cliente')
    print(f"Features (FASE 2) carregadas com sucesso: {df_features.shape}")
except FileNotFoundError:
    print(f"ERRO: Arquivo 'features_clientes.csv' não encontrado em {features_path}")
    print("Execute o notebook 02_engenharia_features.ipynb primeiro.")
    

# 2. Carregar dados brutos (FASE 1) para gerar os labels de churn
df_historico_raw = get_historical_data(dat_start_filter='2024-01-01') # Use o mesmo período da FASE 2


# 3. Gerar os Labels de Churn (FASE 5 - Passo 1)
if df_historico_raw is not None:
    df_churn_labels = create_churn_labels(df_historico_raw, days_for_churn=45)
    print(f"Labels de Churn (FASE 5) gerados com sucesso: {df_churn_labels.shape}")
else:
    print("ERRO: Falha ao carregar dados históricos para gerar labels de churn.")

In [ ]:
# Célula 3: Juntar Features e Labels (A Tabela de Treino)

if 'df_features' in locals() and 'df_churn_labels' in locals():
    
    # Junta as features (X) com o alvo (y)
    df_churn_treino = df_features.join(df_churn_labels)
    
    # Remove clientes para os quais não conseguimos calcular o churn (raro)
    df_churn_treino = df_churn_treino.dropna(subset=['is_churn'])
    
    print("\n--- Tabela de Treino de Churn (Pronta!) ---")
    print(df_churn_treino.head())
    
    print("\nDistribuição de Churn (Clientes Ativos vs. Churn):")
    # Mostra a proporção (0 = Ativo, 1 = Churn)
    print(df_churn_treino['is_churn'].value_counts(normalize=True))
    
    # Salva esta tabela final para o próximo passo (treinamento)
    output_dir = os.path.join(project_root, 'dados', 'processed')
    output_path = os.path.join(output_dir, 'features_churn_clientes.csv')
    
    df_churn_treino.to_csv(output_path)
    print(f"\nTabela de treino de churn salva em: {output_path}")
    
else:
    print("\nERRO: Falha ao carregar features ou labels. A tabela de treino não foi criada.")